In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from numpy.linalg import eig

In [2]:
#path = "C:/Users/Sachith/Desktop/work_oh/bulk_water/MD1"
path='C:/shehani/postdoc_work/ML_Diffusion/md_data/c4_nmeth_nh2o_box1/c2_frozen_48/'

In [3]:
f=open(path +'/input_data.dat','r')
lines=f.readlines()
for line in lines[0:]:
    data=line.split()
    if data[0]=='nsteps:':
        nsteps = int(data[1])
    if data[0]=='natoms:':
        natoms = int(data[1])
    if data[0]=='nonmvat:':
        nonmvat = int(data[1])
    if data[0]=='nmovingatoms:':
        nmovingatoms = int(data[1])
    if data[0]=='MDRestartFrequency:':
        MDRestartFrequency = int(data[1])
    if data[0]=='dt:':
        dt = float(data[1])
    if data[0]=='noh:':
        noh = int(data[1])
    if data[0]=='nwater:':
        nwater = int(data[1])
    if data[0]=='nox:':
        nox = int(data[1])
    if data[0]=='nhy:':
        nhy = int(data[1])
    if data[0]=='xbox:':
        xbox = float(data[1])
    if data[0]=='ybox:':
        ybox = float(data[1])
    if data[0]=='zbox:':
        zbox = float(data[1])
        
nsteps = int((nsteps/MDRestartFrequency)+1)-1
nstep = nsteps
        
print('nsteps:','  ',nstep )
print('natoms:','  ',natoms)
print('nonmvat:','  ',nonmvat)
print('nmovingatoms:','  ',nmovingatoms)
print('MDRestartFrequency:','  ',MDRestartFrequency)
print('dt:','  ',dt)
print('noh:','  ',noh)
print('nwater:','  ',nwater)
print('nox:','  ',nox)
print('nhy:','  ',nhy)
print('xbox:','  ',xbox)
print('ybox:','  ',ybox)
print('zbox:','  ',zbox)

nsteps:    1000
natoms:    452
nonmvat:    306
nmovingatoms:    146
MDRestartFrequency:    10
dt:    1.0
noh:    1
nwater:    48
nox:    49
nhy:    97
xbox:    15.198
ybox:    13.392
zbox:    15.0


In [4]:
msd_of_oh = np.empty(nstep)
msd_of_ohx = np.empty(nstep)
msd_of_ohy = np.empty(nstep)
msd_of_ohz = np.empty(nstep)
oh_index = np.empty(nstep)
xoh = np.empty(nstep)
yoh = np.empty(nstep)
zoh = np.empty(nstep)
atom= ['Non']*(natoms)
x_ox = np.empty(nstep*nox).reshape(nox,nstep)
y_ox = np.empty(nstep*nox).reshape(nox,nstep)
z_ox = np.empty(nstep*nox).reshape(nox,nstep)

x_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
y_hy = np.empty(nstep*nhy).reshape(nhy,nstep)
z_hy = np.empty(nstep*nhy).reshape(nhy,nstep)

In [5]:
cat=0
jj=0
kk=0
f=open(path +'/geo_end.xyz','r')
lines=f.readlines()
while jj < nstep:
    io=0
    ih=0
    for line in lines[kk+nonmvat+cat+2:(kk+natoms+2)]:
        data=line.split()
        if len(data)>3:
            if data[0]=='O':
                x_ox[io][jj]=float(data[1])
                y_ox[io][jj]=float(data[2])
                z_ox[io][jj]=float(data[3])
                io = io+1
            elif data[0]=='H':
                x_hy[ih][jj]=float(data[1])
                y_hy[ih][jj]=float(data[2])
                z_hy[ih][jj]=float(data[3])
                ih = ih+1
    kk=kk+natoms+2
    jj=jj+1
f.close()

In [6]:
nrep = nox
nsteps = 1000
aa = [24,42,49,63,18,22,5,25,69,26,20,10,71,1,39,68,51,8,19,55,64,9,60,52]
x_o = np.empty(nsteps*nrep).reshape(nrep,nsteps)
y_o = np.empty(nsteps*nrep).reshape(nrep,nsteps)
z_o = np.empty(nsteps*nrep).reshape(nrep,nsteps)

In [7]:
kk = 0
while (kk<nox):
    jj = 0
    while (jj<1000):
        x_o[kk][jj] = x_ox[kk][jj] 
        y_o[kk][jj] = y_ox[kk][jj] 
        z_o[kk][jj] = z_ox[kk][jj] 
        jj = jj+1
    kk = kk+10


In [8]:
ndt = 50
mmsdx = np.empty(ndt)
mmsdy = np.empty(ndt)
mmsdz = np.empty(ndt)
mmsdxy = np.empty(ndt)
mmsdxz = np.empty(ndt)
mmsdyz = np.empty(ndt)
dtime = np.empty(ndt)
dtime=  np.arange(0.01, (0.01*ndt+0.01), 0.01)

Dfxx = np.empty(nrep)  
Dfyy = np.empty(nrep)
Dfzz = np.empty(nrep)
Dfxy = np.empty(nrep)
Dfxz = np.empty(nrep)
Dfyz = np.empty(nrep)
Dfx2d = np.empty(nrep)
Dfy2d = np.empty(nrep)
Df2d = np.empty(nrep)

ll = 0
while ll<nox:
    kk = 0
    while kk<ndt:
        mmsdx[kk] = 0.0
        mmsdy[kk] = 0.0
        mmsdz[kk] = 0.0
        mmsdxy[kk] = 0.0
        mmsdxz[kk] = 0.0
        mmsdyz[kk] = 0.0
        jj = 0
        while jj< ((nsteps/(kk+1))-1):
            mmsdx[kk] = (x_o[ll,(jj*(kk+1)+kk+1)]-x_o[ll,jj*(kk+1)])**2+mmsdx[kk]
            mmsdy[kk] = (y_o[ll,(jj*(kk+1)+kk+1)]-y_o[ll,jj*(kk+1)])**2+mmsdy[kk]
            mmsdz[kk] = (z_o[ll,(jj*(kk+1)+kk+1)]-z_o[ll,jj*(kk+1)])**2+mmsdz[kk]
            mmsdxy[kk] = ((x_o[ll,(jj*(kk+1)+kk+1)]-x_o[ll,jj*(kk+1)])*(y_o[ll,(jj*(kk+1)+kk+1)]-y_o[ll,jj*(kk+1)]))+mmsdxy[kk]
            mmsdxz[kk] = ((x_o[ll,(jj*(kk+1)+kk+1)]-x_o[ll,jj*(kk+1)])*(z_o[ll,(jj*(kk+1)+kk+1)]-z_o[ll,jj*(kk+1)]))+mmsdxz[kk]
            mmsdyz[kk] = ((y_o[ll,(jj*(kk+1)+kk+1)]-y_o[ll,jj*(kk+1)])*(z_o[ll,(jj*(kk+1)+kk+1)]-z_o[ll,jj*(kk+1)]))+mmsdyz[kk]
            
            jj = jj+1
    
        mmsdx[kk] = mmsdx[kk]/(((nsteps/(kk+1))-1))
        mmsdy[kk] = mmsdy[kk]/(((nsteps/(kk+1))-1))
        mmsdz[kk] = mmsdz[kk]/(((nsteps/(kk+1))-1))
        mmsdxy[kk] = mmsdxy[kk]/(((nsteps/(kk+1))-1))
        mmsdxz[kk] = mmsdxz[kk]/(((nsteps/(kk+1))-1))
        mmsdyz[kk] = mmsdyz[kk]/(((nsteps/(kk+1))-1))
        print(ll, mmsdx[kk],nsteps,(kk+1),(((nsteps/(kk+1))-1)))
        kk = kk+1
    
    
    fitx=np.polyfit(dtime[:],mmsdx[:],1)
    fity=np.polyfit(dtime[:],mmsdy[:],1)
    fitz=np.polyfit(dtime[:],mmsdz[:],1)
    fitxy=np.polyfit(dtime[:],mmsdxy[:],1)
    fitxz=np.polyfit(dtime[:],mmsdxz[:],1)
    fityz=np.polyfit(dtime[:],mmsdyz[:],1)

    dxx = fitx[0]/2
    dyy = fity[0]/2
    dzz = fitz[0]/2
    dxy = fitxy[0]/2
    dxz = fitxz[0]/2
    dyz = fityz[0]/2


    dd2d = np.array([[dxx, dxy], 
                 [dxy, dyy]])
    w2,v2=eig(dd2d)


    Dfxx[ll] = dxx
    Dfyy[ll] = dyy
    Dfzz[ll] = dzz
    Dfxy[ll] = dxy
    Dfxz[ll] = dxz
    Dfyz[ll] = dyz
    Dfx2d[ll] = w2[0]
    Dfy2d[ll] = w2[1]
    Df2d[ll] = ((w2[0]+w2[1])/2)
    
#    print(ll ,Dfxx[ll])
    ll = ll+1



0 0.002047070715385336 1000 1 999.0
0 0.007963035029714877 1000 2 499.0
0 0.017580613035629536 1000 3 332.3333333333333
0 0.03012996170540806 1000 4 249.0
0 0.046192913631989914 1000 5 199.0
0 0.0629930125160671 1000 6 165.66666666666666
0 0.08403915126518462 1000 7 141.85714285714286
0 0.10534258851344654 1000 8 124.0
0 0.1293011032864339 1000 9 110.11111111111111
0 0.14723361738318608 1000 10 99.0
0 0.17087485964018873 1000 11 89.9090909090909
0 0.2052823334822737 1000 12 82.33333333333333
0 0.21313737131370097 1000 13 75.92307692307692
0 0.22557775127927118 1000 14 70.42857142857143
0 0.2793425982040935 1000 15 65.66666666666667
0 0.27253781052972725 1000 16 61.5
0 0.3433882962834439 1000 17 57.8235294117647
0 0.3489302814989036 1000 18 54.55555555555556
0 0.38572262696772985 1000 19 51.63157894736842
0 0.42675907946609154 1000 20 49.0
0 0.4406474520643287 1000 21 46.61904761904762
0 0.4801262342347671 1000 22 44.45454545454545
0 0.5570805427449044 1000 23 42.47826086956522
0 0.5647

In [9]:
print('Dxx  =','{0: >#014.10f}'.format(np.mean(Dfxx)),'{0: >#014.10f}'.format(np.std(Dfxx)/np.sqrt(nrep)))
print('Dyy  =','{0: >#014.10f}'.format(np.mean(Dfyy)),'{0: >#014.10f}'.format(np.std(Dfyy)/np.sqrt(nrep)))
print('Dxy  =','{0: >#014.10f}'.format(np.mean(Dfxy)),'{0: >#014.10f}'.format(np.std(Dfxy)/np.sqrt(nrep)))
print('-----------------------')
print('Dx2  =','{0: >#014.10f}'.format(np.mean(Dfx2d)),'{0: >#014.10f}'.format(np.std(Dfx2d)/np.sqrt(nrep)))
print('Dy2  =','{0: >#014.10f}'.format(np.mean(Dfy2d)),'{0: >#014.10f}'.format(np.std(Dfy2d)/np.sqrt(nrep)))
print('D2d  =','{0: >#014.10f}'.format(np.mean(Df2d)),'{0: >#014.10f}'.format(np.std(Df2d)/np.sqrt(nrep)))

Dxx  =   0.1689556827   0.0729344882
Dyy  =   0.1376221835   0.0596603254
Dxy  =   0.0206907823   0.0166229098
-----------------------
Dx2  =   0.1637580723   0.0730049290
Dy2  =   0.1428197939   0.0642241004
D2d  =   0.1532889331   0.0653889642


In [10]:
for item in Dfxx:
    print(item)

1.2482244635186506
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.1686087995458747
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.6026239883908908
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.5079335594508432
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.751437641533577
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
